<a href="https://colab.research.google.com/github/margotbrady/DS-1002/blob/main/BradyAPIHomework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import necessary libraries
import requests
import json
import pandas as pd

In [71]:
stock= input("Input the ticker symbol you would like to research: ")

Input the ticker symbol you would like to research: AAPL


In [74]:
def getstockinfo(symbol):
    #Get info from finance/quote
    apikey="generickey"
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols":stock}
    headers = {
        'x-api-key': apikey
        }
    # genresponse = requests.request("GET", url, headers=headers, params=querystring)

    try:
        genresponse = requests.get(url, headers=headers, params=querystring)
        genresponse.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.InvalidHeader as erri:
        return "A Header Error occurred: " + repr(erri)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)


    #Get info further details
    url2 = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    detailquery = {"modules": "price,summaryDetail,defaultKeyStatistics"}
    headers = {
        'x-api-key': apikey
        }
    detailinfo = requests.request("GET",url2, headers = headers, params = detailquery)

    # Get 5 trending stocks

    url3 = "https://yfapi.net/v1/finance/trending/us"

    # No need for query parameters for trending stocks in this case
    headers = {
        'x-api-key': apikey
    }

    # Make the request
    trendresponse = requests.request("GET", url3, headers=headers)

    #Choose 5 trending stocks from the response:
    trendingstocks = trendresponse.json()["finance"]["result"][0]["quotes"][:5]
    stock_symbols = [stock['symbol'] for stock in trendingstocks]
    #Get info from the API calls
    genstock_json = genresponse.json()
    detailstock_json = detailinfo.json()

    try:
      print("Ticker Name: " + symbol +
          "\nStock Name: "+genstock_json['quoteResponse']['result'][0]["longName"]+
          "\nCurrent Market Price: $" + str(genstock_json['quoteResponse']['result'][0]["regularMarketPrice"])+
          #Target Mean Price = Target Mean Price=Forward EPS×Forward P/E Ratio
          "\nTarget Mean Price: $" + str(detailstock_json['quoteSummary']['result'][0]['defaultKeyStatistics']['forwardEps']['raw'] *detailstock_json['quoteSummary']['result'][0]['defaultKeyStatistics']["forwardPE"]['raw'])+
          "\n52 Week High: $" + str(genstock_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"])+
          "\n52 Week Low: $" + str(genstock_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"])+
          "\nTrending Stocks: " + str(stock_symbols))
    except KeyError:
        print(f"Error: Stock symbol '{symbol}' not found.")
        return

    # Turn data into a dataframe:

    data = {
        "Ticker Name": [symbol],
        "Stock Name": [genstock_json['quoteResponse']['result'][0]["longName"]],
        "Current Market Price": [genstock_json['quoteResponse']['result'][0]["regularMarketPrice"]],
        "Target Mean Price": [detailstock_json['quoteSummary']['result'][0]['defaultKeyStatistics']['forwardEps']['raw'] *detailstock_json['quoteSummary']['result'][0]['defaultKeyStatistics']["forwardPE"]['raw']],
        "52 Week High": [genstock_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"]],
        "52 Week Low": [genstock_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"]],
        "Trending Stocks": [stock_symbols]
    }

    df1 = pd.DataFrame(data)
# store that to a CSV file
    df1.to_csv("stock_info.csv", index=False)

In [75]:
#Ask for Ticker Symbol
getstockinfo(stock)

Input the ticker symbol you would like to research: AAPL
Ticker Name: AAPL
Stock Name: Apple Inc.
Current Market Price: $227.725
Target Mean Price: $227.72500212
52 Week High: $237.23
52 Week Low: $164.08
Trending Stocks: ['NQ=F', 'ES=F', 'YM=F', 'AMSC', 'GC=F']
